I will be testing biomed clip with a little set of images, just searching the elements of calcifications and nodules (maybe include the birads) to make the radiological report. 

In [1]:
from open_clip import create_model_from_pretrained, get_tokenizer # works on open-clip-torch>=2.23.0, timm>=0.9.8
tokenizer = get_tokenizer('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from PIL import Image
import os
import json
import pandas as pd
import numpy as np
import nrrd
from torchvision import transforms
from typing import List, Optional
from transformers import TrainingArguments, Trainer


In [4]:
def fine_tune_biomed_clip(model, train_dataloader, num_epochs, device, learning_rate=5e-5):
    """
    Manual training loop for BioMed CLIP 
    THIS CODE HAS NOT BEEN TESTED
    """
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        num_batches = 0
        
        for batch in train_dataloader:
            try:
                # Move batch to device
                images = batch['image'].to(device)
                texts = batch['text']
                
                if isinstance(texts, dict):
                    texts = {k: v.to(device) for k, v in texts.items()}
                else:
                    texts = texts.to(device)
                
                # Forward pass
                outputs = model(image=images, text=texts)
                loss = outputs.loss if hasattr(outputs, 'loss') else outputs[0]
                
                # Backward pass and optimization
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                
                # Accumulate loss
                total_loss += loss.item()
                num_batches += 1
                
                if num_batches % 10 == 0:
                    print(f"Epoch {epoch+1}/{num_epochs}, Batch {num_batches}, Loss: {loss.item():.4f}")
                
            except Exception as e:
                print(f"Error in batch: {str(e)}")
                continue
        
        # Calculate and print average loss for the epoch
        avg_loss = total_loss / num_batches
        print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")
    
    return model

In [5]:
def generate_text(model, processor, image_paths, entity_info=None, max_length=100):
    """
    Generate text descriptions from medical images using BioMedCLIP.
    THIS CODE HAS NOT BEEN TESTED, I DONT KNOW IF IT WORKS
    """
    # Load and process all images
    images = [Image.open(path) for path in image_paths]
    
    # Process images with the BioMedCLIP processor
    inputs = processor(images, return_tensors="pt", padding=True)
    
    # Move inputs to the same device as model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    # Generate text without gradient computation
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=max_length)
    
    # Decode the generated text
    generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0]
    
    return generated_text

In [ ]:
from dataset_pl import MyDatamodule
from model_pl import MyModel
import lightning as L

from transformers import AdamW

import torch
from torch.utils.data import DataLoader
from open_clip import create_model_from_pretrained, get_tokenizer

"""
BioMedCLIP Setup and Data Loading Script

This script initializes the BioMedCLIP model, preprocessor, and tokenizer,
then sets up the dataset and dataloader for training or inference.

"""

# Initialize BioMedCLIP model and preprocessor
model, preprocess = create_model_from_pretrained('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')

# Initialize tokenizer
tokenizer = get_tokenizer('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')

# Configuration parameters
CONFIG = {
    'batch_size': 32,
    'num_workers': 4,
    'model_name': 'microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224',
    'data_dir': "/Users/YusMolina/Documents/tesis/biomedCLIP/data/datosMex/",
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu')
}

# Create datamodule instance
'''
En vez de crear un dataset, creamos una instancia del datamodule, que contendrá el dataset y los dataloaders.
dataset = ComplexMedicalDataset(
    data_dir="/Users/YusMolina/Documents/tesis/biomedCLIP/data/datosMex/",
    processor=preprocess,
    tokenizer=tokenizer
)

# Create DataLoader
dataloader = DataLoader(
    dataset, 
    batch_size=32, 
    shuffle=True, 
    collate_fn=ComplexMedicalDataset.collate_fn
    )
'''
datamodule = MyDatamodule(
    data_dir="/Users/YusMolina/Documents/tesis/biomedCLIP/data/datosMex/",
    processor=preprocess,
    tokenizer=tokenizer,
    batch_size=32,
    num_workers=4)

# Create an instance of MyModel
model = MyModel(
    model=model,
    optimizer=AdamW(model.parameters(), lr=5e-5),
    loss="<TODO: Loss function here>"
    )

trainer = L.trainer()

trainer.train(model, datamodule)
trainer.eval(model, datamodule)

"""
Expected directory structure:
/data/datosMex/
    ├── dataset_info.json
    ├── images/
    │   ├── image1.jpg
    │   ├── image2.jpg
    │   └── ...
    └── reports/
        ├── report1.txt
        └── ...

dataset_info.json format:
{
    "case_id": {
        "image_paths": ["path/to/image1.jpg", ...],
        "report": "medical report text"
    },
    ...
}
"""

# Optional: Print configuration for verification
def print_setup_info():
    """Print configuration and setup information for verification."""
    print("\nBioMedCLIP Setup Information:")
    print(f"Device: {CONFIG['device']}")
    print(f"Dataset size: {len(dataset)}")
    print(f"Number of batches: {len(dataloader)}")
    print(f"Batch size: {CONFIG['batch_size']}")
    print(f"Number of workers: {CONFIG['num_workers']}")
    print("\nModel Information:")
    print(f"Model name: {CONFIG['model_name']}")
    


taken from the tutorial: https://huggingface.co/docs/transformers/training

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [9]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [10]:
import numpy as np
import torch

def compute_metrics(eval_pred):
    """
    Compute evaluation metrics for CLIP-like models, specifically designed for
    image-text matching tasks.
    
    NOT TESTED
    """
    outputs = eval_pred.predictions
    
    # Handle CLIP-style embeddings (image_embeds, text_embeds)
    if isinstance(outputs, tuple):
        image_embeds, text_embeds = outputs
        
        # Compute cosine similarity matrix
        similarity = torch.matmul(
            torch.from_numpy(image_embeds), 
            torch.from_numpy(text_embeds).t()
        )
        
        # Get predictions (diagonal elements should be highest for correct pairs)
        predictions = torch.argmax(similarity, dim=1).numpy()
        labels = np.arange(len(predictions))  # Assuming paired data
        
        # Calculate evaluation metrics
        accuracy = (predictions == labels).mean()
        mean_similarity = similarity.diagonal().mean().item()
        
        # Return dictionary of metrics
        return {
            "accuracy": accuracy,                    # Matching accuracy
            "mean_similarity": mean_similarity       # Average similarity for correct pairs
        }
    
    # Handle standard classification outputs
    else:
        predictions = np.argmax(outputs, axis=-1)
        labels = eval_pred.label_ids
        accuracy = (predictions == labels).mean()
        return {"accuracy": accuracy}




In [11]:
# I know is bad, sorry
train_dataset = dataset
eval_dataset = dataset

In [12]:
# Training arguments setup
def setup_training(
    output_dir: str = "./results",
    num_epochs: int = 3,
    eval_steps: int = 100,
    learning_rate: float = 5e-5
) -> TrainingArguments:
   
    return TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_epochs,        
        eval_steps=eval_steps,
        learning_rate=learning_rate,
        metric_for_best_model="accuracy",
        logging_dir="./logs",
        logging_steps=10,
        report_to="tensorboard",
        # Gradient related
        gradient_accumulation_steps=1,
        weight_decay=0.01,
        # Mixed precision training
        fp16=torch.cuda.is_available(),
        # Other settings
        remove_unused_columns=False,  # Important for CLIP-like models
    )

In [13]:
training_args = setup_training(output_dir="./results")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= train_dataset,
    eval_dataset= eval_dataset,
    compute_metrics=compute_metrics,
)

Prints to obain more information of the model 

In [ ]:
trainer.train()

In [ ]:
help(model.forward)


In [ ]:
import torchinfo

torchinfo.summary(model, input_size=(4, 3, 224, 224))  # specify input size


In [ ]:
model